[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/duckietown/challenge-aido_LF-baseline-dagger-pytorch/blob/main/notebook.ipynb)

## Preparing Code and dependencies

In [5]:
branch = "daffy" #@param ['master', 'daffy']
!apt-get install python3.8
!apt install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3.8 get-pip.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.8 is already the newest version (3.8.8-1+bionic2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.8-distutils'
python3-distutils is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
--2021-03-13 17:55:25--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1927626 (1.8M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   1.84M  --.-KB/s    in 0.03s   

2021-03-13 17:55:25 (60.5 MB/s) - ‘get-pip.py

In [6]:
!pip3.8 freeze

absl-py==0.12.0
aido-protocols-daffy==6.0.33
asciimatics==1.12.0
astunparse==1.6.3
attrs==20.3.0
base58==1.0.3
beautifulsoup4==4.7.1
cachetools==4.2.1
carnivalmirror==0.6.2
cbor2==5.2.0
certifi==2020.12.5
chardet==4.0.0
cloudpickle==1.6.0
colorama==0.4.4
coloredlogs==15.0
compmake-z6==6.1.0
coverage==5.5
cycler==0.10.0
dataclasses==0.6
dataclasses-json==0.5.2
decorator==4.4.2
dill==0.3.3
-e git+https://github.com/duckietown/gym-duckietown@2a0693c02cc2f42de30ef08221c77611af378c6d#egg=duckietown_gym_daffy
duckietown-serialization-ds1==1.0.10
duckietown-world-daffy==6.2.7
EasyProcess==0.3
freetype-py==2.2.0
frozendict==1.2
future==0.18.2
gltflib==1.0.5
google-auth==1.27.1
google-auth-oauthlib==0.4.3
grpcio==1.36.1
gym==0.18.0
humanfriendly==9.1
idna==2.10
imageio==2.9.0
jsonschema==3.2.0
kiwisolver==1.3.1
lxml==4.6.2
Markdown==3.3.4
MarkupSafe==1.1.1
marshmallow==3.10.0
marshmallow-enum==1.5.1
matplotlib==3.3.4
morphys==1.0
mypy-extensions==0.4.3
networkx==2.5
nose==1.3.7
numpy==1.20.1
oa

In [7]:
! git clone --branch {branch} https://github.com/duckietown/gym-duckietown

fatal: destination path 'gym-duckietown' already exists and is not an empty directory.


In [8]:
!pip3.8 install -e gym-duckietown/.

Obtaining file:///content/gym-duckietown
  Attempting uninstall: duckietown-gym-daffy
    Found existing installation: duckietown-gym-daffy 6.1.1
    Can't uninstall 'duckietown-gym-daffy'. No files were found to uninstall.
  Running setup.py develop for duckietown-gym-daffy


In [9]:
! git clone https://github.com/duckietown/challenge-aido_LF-baseline-dagger-pytorch

fatal: destination path 'challenge-aido_LF-baseline-dagger-pytorch' already exists and is not an empty directory.


In [26]:
import os 
os.chdir('challenge-aido_LF-baseline-dagger-pytorch')

### Virtual Display Setup

In [27]:
!apt-get install python-opengl -y
!apt install xvfb -y
!apt-get install x11-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [29]:
!pip3.8 install pyvirtualdisplay
!pip3.8 install piglet

In [3]:
from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
def create_display():
    display = Display(visible=0, size=(1400, 900))
    display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !bash ../xvfb start
        %env DISPLAY=:1

## Imitation Learning Dagger

## Training

In [31]:
learning_rates = ['1e-1', '1e-2', '1e-3', '1e-4', '1e-5']
mixing_decays = ['0.5', '0.6', '0.7', '0.8', '0.85', '0.9', '0.95']

In [32]:
save_path = "imitation_baseline" #@param {type: "string"}
episode = 10 # @param {type: "integer"}
horizon = 128 # @param {type: "integer"}
learning_rate = "1e-3" # @param ['1e-1', '1e-2', '1e-3', '1e-4', '1e-5']
decay = "0.7" # @param ['0.5', '0.6', '0.7', '0.8', '0.85', '0.9', '0.95']
map_name = "loop_empty" #@param {type: "string"}
# number of outputs can be 2 to predict omega and velocity
# or  1 to fix velocity and predict only omega
num_outputs = 2 # @param {type: "integer"} 
learning_rate = learning_rates.index(learning_rate)
decay = mixing_decays.index(decay)


In [42]:
!pip3.8 install tqdm torch torchvision tensorboard

     |████████████████████████████████| 10.6 MB 19.6 MB/s 
     |████████████████████████████████| 781 kB 61.8 MB/s 
     |████████████████████████████████| 61 kB 9.3 MB/s 
     |████████████████████████████████| 1.0 MB 68.2 MB/s 
     |████████████████████████████████| 298 kB 71.4 MB/s 
     |████████████████████████████████| 129 kB 83.3 MB/s 
     |████████████████████████████████| 4.1 MB 53.7 MB/s 
     |████████████████████████████████| 136 kB 77.2 MB/s 
     |████████████████████████████████| 155 kB 86.5 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 58 kB 8.3 MB/s 
     |████████████████████████████████| 137 kB 90.1 MB/s 
     |████████████████████████████████| 147 kB 89.3 MB/s 
     |████████████████████████████████| 178 kB 73.4 MB/s 
     |████████████████████████████████| 147 kB 95.3 MB/s 


In [10]:
create_display()
!python3.8 -m learning.train --save-path {save_path} --episode {episode} --horizon {horizon} --learning-rate {learning_rate} --decay {decay} --map-name {map_name} --num-outputs {num_outputs}

/usr/bin/python3.8: Error while finding module specification for 'learning.train' (ModuleNotFoundError: No module named 'learning')


### View on Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {save_path}

## Testing Imitation Model

In [ ]:
map_name = "loop_empty" #@param {type: "string"}
episode = 10 # @param {type: "integer"}
horizon = 128 # @param {type: "integer"}

In [ ]:
create_display()
!python3.8 -m learning.test --model-path {os.path.join(save_path, "model.pt")} --num-outputs {num_outputs} --map-name {map_name} --episode {episode} --horizon {horizon}